<div style="
    font-family: 'Georgia', serif;
    color: #2c2c2c;
    text-align: center;
    padding: 20px 0 30px 0;
">

  <h2 style="
      font-size: 26px;
      color: #ad1457;
      font-weight: bold;
      letter-spacing: 0.5px;
      border-bottom: 2px solid #f48fb1;
      padding-bottom: 6px;
      margin: 0 auto 10px auto;
      width: fit-content;
      min-width: 300px;
  ">
    Collecte des données de projections climatiques
  </h2>

  <p style="font-size: 15px; margin: 8px 0 2px 0;">
    <strong>Hajar BADRAOUI</strong> 
  </p>

  <p style="font-size: 14px; margin-top: 10px;">
    📧 <a href="mailto:hajar.badraoui01@gmail.com" style="color: #ad1457; text-decoration: none;">
      hajar.badraoui01@gmail.com
    </a>
  </p>

  <p style="font-size: 14px; margin-top: 5px;">
    🔗 <a href="https://github.com/badraouihajar" target="_blank" style="color: #ad1457; text-decoration: none;">
      github.com/badraouihajar
    </a>
  </p>

</div>


<div style="
    background-color: #fff;
    padding: 26px 30px 18px 30px;
    border-radius: 12px;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.06);
    font-family: 'Georgia', serif;
    color: #2c2c2c;
    margin: 12px 0 20px 0;
">

<h3 style="margin-top:0; margin-bottom:18px; color:#d81b60;">
  Introduction – Données & Modèles climatiques
</h3>
<p>
Tout d'abord, on commence par télécharger et prétraiter les données des modèles climatiques dans le but de corriger les biais.
</p>

<p>
Pour appliquer une méthode de correction de biais, il est nécessaire de disposer de <b>trois ensembles de données</b> :
<ul style="margin-top:8px;">
  <li><b>Données d'observation ou de réanalyse</b></li>
  <li><b>Données historiques issues des modèles climatiques</b> (même période que les observations)</li>
  <li><b>Données des modèles climatiques pour une période future</b></li>
</ul>
</p>

<p>
Dans cette étude, nous allons travailler avec des <b>modèles climatiques EURO-CORDEX</b> afin d'analyser les évolutions climatiques passées et futures selon le scénario <b>RCP 4.5</b>.
</p>

<p style="margin-bottom:4px;"><b>Modèles climatiques utilisés</b> :</p>
<ul>
  <li>Modèle global <b>CNRM-CERFACS-CM5</b> (France) / modèle régional <b>CNRM-ALADIN63</b> (France)</li>
  <li>Modèle global <b>CNRM-CERFACS-CM5</b> (France) / modèle régional <b>KNMI-RACMO22E</b> (Pays-Bas)</li>
  <li>Modèle global <b>ICHEC-EC-EARTH</b> (Irlande) / modèle régional <b>KNMI-RACMO22E</b> (Pays-Bas)</li>
</ul>

<p style="margin-bottom:4px;"><b>Périodes et scénarios étudiés</b> :</p>
<ul>
  <li><b>Période historique</b> : 1976 – 2005</li>
  <li><b>Scénario RCP 4.5</b> : 2006 – 2100 (scénario modéré d’émissions)</li>
</ul>

<p style="margin-bottom:4px;"><b>Données de réanalyse</b> :</p>
<ul>
  <li><b>SAFRAN</b> : données de réanalyse utilisées comme référence pour valider les simulations climatiques.</li>
</ul>
</div>

le package Iris est un outil Python spécialisé pour la lecture, manipulation et analyse des données climatologiques et météorologiques au format NetCDF.

In [ ]:
!pip install scitools-iris   

In [2]:
import os
import urllib
import iris
import xarray
import numpy as np

<h2 style="font-family: 'Alice', serif; color: #2471A3; font-size: 2em; font-weight: 700;">
  Étape 01 : Collecte des données
</h2>

#### <span style="color:#2980b9;;">1.1 Téléchargement des données du modèle climatique</span>

Pour demander des données climatiques à partir du Climate Data Store (CDS), nous utiliserons l’API CDS.

In [ ]:
!pip install cdsapi

Dans l’ensemble des scripts présentés, le traitement est illustré avec le modèle régional ALADIN (CNRM-ALADIN63), mais la même démarche peut être appliquée aux autres modèles climatiques utilisés dans cette étude (ex : KNMI-RACMO22E, ICHEC-EC-EARTH)

#### <span style="color:#6A5ACD;">a. Téléchargement des données du modèle climatique historique</span>

In [1]:
import cdsapi
import urllib3
urllib3.disable_warnings()

Le code ci-dessous permet de télécharger les fichiers `.zip` directement depuis le site [Copernicus CDS](https://cds.climate.copernicus.eu/datasets/projections-cordex-domains-single-levels?tab=download) pour le jeu de données EURO-CORDEX.

In [ ]:
# Le code n'est pas à modifier, **sauf** pour les éléments suivants :
# === Les seules choses à changer sont :
#   - gcm_model    : nom du modèle global ( cnrm_cerfacs_cm5  , ichec_ec_earth  )
#   - rcm_model   :  nom du modèle régional (knmi_racmo22e ,  knmi_racmo22e )
#   - experiment  : nom du scénario (ex : 'historical', 'rcp_4_5', 'rcp_8_5') (dans ce code est dédié que pour hist, sinon on doit changer les années)
# A RETENIR le dossier destination folder : le dossier qui va contenir les fichiers netcf du modèle historique
# Tout le reste du code doit rester inchangé.


import cdsapi
import os
import time
import sys
 
#Désactiver les widgets Jupyter (évite le bug de blocage)
sys.modules["ipywidgets"] = None
 
#Dossier de destination
destination_folder ="D:/cordex_data/donnees_historiques"        # à VOIR 
os.makedirs(destination_folder, exist_ok=True)
 
#Initialisation de l'API
client = cdsapi.Client()
 
#Paramètres du dataset
dataset = "projections-cordex-domains-single-levels"
experiment = "historical"                                 # à MODIFIER
gcm_model = "cnrm_cerfacs_cm5"                            # à MODIFIER
rcm_model = "cnrm_aladin63"                               # à MODIFIER
ensemble_member = "r1i1p1"
horizontal_resolution = "0_11_degree_x_0_11_degree"
temporal_resolution = "daily_mean"
 
#Variables climatiques
variables = [
    "2m_relative_humidity",
    "10m_wind_speed",
    "maximum_2m_temperature_in_the_last_24_hours",
    "minimum_2m_temperature_in_the_last_24_hours",
    "mean_precipitation_flux",
    "surface_solar_radiation_downwards"
]
 
#Périodes à télécharger
periods = [
    ("1951", "1955"), ("1956", "1960"), ("1961", "1965"),
    ("1966", "1970"), ("1971", "1975"), ("1976", "1980"),
    ("1981", "1985"), ("1986", "1990"), ("1991", "1995"),
    ("1996", "2000"), ("2001", "2005"),
]
 
#Téléchargement par période
for i, (start_year, end_year) in enumerate(periods):
    filename = f"{destination_folder}/cordex_{experiment}_{gcm_model}_{rcm_model}_{start_year}_{end_year}.zip"
 
    print(f"\n Bloc {i+1}/{len(periods)} : {start_year}-{end_year}")
 
    #  Ne pas re-télécharger si déjà présent
    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        print(f" Déjà téléchargé : {filename}")
        continue
 
    print(f" Téléchargement en cours...")
 
    request = {
        "domain": "europe",
        "experiment": experiment,
        "horizontal_resolution": horizontal_resolution,
        "temporal_resolution": temporal_resolution,
        "variable": variables,
        "gcm_model": gcm_model,
        "rcm_model": rcm_model,
        "ensemble_member": ensemble_member,
        "start_year": [start_year],
        "end_year": [end_year],
        "format": "zip" 
    }
 
    try:
        result = client.retrieve(dataset, request)
        print(" Téléchargement terminé. Sauvegarde du fichier (en mode console)...")
        path = result.download(filename)
        print(f" Fichier enregistré : {path}")
        time.sleep(1)
    except Exception as e:
        print(f" Erreur pour {start_year}-{end_year} : {e}")
 
print("\n Tous les blocs ont été traités avec succès.")

#### <span style="color:#6A5ACD;">b. Extraction géographique : Zone France</span>

Cette section présente un script permettant d’extraire automatiquement **les données climatiques historiques du modèle CNRM-CERFACS-CM5-CNRM-ALADIN63**, en nous concentrant uniquement sur le **territoire français** (même chose pour les autres modèles).

L’objectif est de parcourir l’ensemble des fichiers NetCDF disponibles, de sélectionner la zone correspondant aux limites géographiques de la France, puis de sauvegarder ces sous-ensembles pour une analyse ciblée à l’échelle nationale.

Le script ci-dessous permet d’automatiser cette extraction pour tous les fichiers du répertoire :

In [ ]:
# Le code doit rester inchangé, à l’exception de la variable `input_folder`.
#  Il faut spécifier un sous-dossier dans le dossier des données historiques du modèle,
# nommé selon le modèle utilisé.
# Exemple : si le dossier principal est `donnees_historiques` et le modèle est
# `CNRM-CERFACS-CM5_CNRM-ALADIN63`, alors :
#     input_folder = 'donnees_historiques/CNRM-CERFACS-CM5_CNRM-ALADIN63'

import os
import xarray as xr
import numpy as np

# Paramètres
input_folder = "D:/cordex_data/donnees_historiques/CNRM-CERFACS-CM5_CNRM-ALADIN63"    # changer CNRM-CERFACS-CM5_CNRM-ALADIN63 par Modèle qu'on veut
output_folder = os.path.join(input_folder, "france_extrait")
os.makedirs(output_folder, exist_ok=True)

# Limites géographiques de la France
lat_min, lat_max = 41, 51
lon_min, lon_max = -5, 10

# Boucle sur tous les fichiers NetCDF du dossier
for file in os.listdir(input_folder):
    if file.endswith(".nc"):
        path = os.path.join(input_folder, file)
        print(f"\n Traitement : {file}")
        try:
            ds = xr.open_dataset(path)

            # Lecture des coordonnées lat/lon
            lat2d = ds["lat"]
            lon2d = ds["lon"]

            # Masque pour la France
            mask = (lat2d >= lat_min) & (lat2d <= lat_max) & (lon2d >= lon_min) & (lon2d <= lon_max)
            iy, ix = np.where(mask)

            if iy.size == 0 or ix.size == 0:
                print(" Aucun point dans la zone France.")
                continue

            ymin, ymax = iy.min(), iy.max()
            xmin, xmax = ix.min(), ix.max()

            # Identifier la variable principale (hors lat/lon/time_bounds)
            var_names = [v for v in ds.data_vars if v not in ['lat', 'lon', 'time_bounds']]
            if not var_names:
                print(" Pas de variable principale trouvée.")
                continue

            var_name = var_names[0]
            print(f" Variable détectée : {var_name}")

            # Extraire les données pour la France
            data_subset = ds[var_name].isel(x=slice(xmin, xmax + 1), y=slice(ymin, ymax + 1))
            lat_subset = lat2d.isel(y=slice(ymin, ymax + 1), x=slice(xmin, xmax + 1))
            lon_subset = lon2d.isel(y=slice(ymin, ymax + 1), x=slice(xmin, xmax + 1))

            # Réassigner les coordonnées
            data_subset = data_subset.assign_coords(lat=lat_subset, lon=lon_subset)

            # Sauvegarde dans le dossier 'france_extrait'
            output_path = os.path.join(output_folder, f"{var_name}_FR_{file}")
            data_subset.to_netcdf(output_path)
            print(f" Sauvegardé : {output_path}")

        except Exception as e:
            print(f" Erreur pour {file} : {e}")

#### <span style="color:#6A5ACD;">c. Téléchargement des données du modèle climatique future</span>

Nous appliquons les mêmes configurations de modèles que précédemment, cette fois dans le cadre du scénario **RCP 4.5**

In [ ]:
# Le code n'est pas à modifier, **sauf** pour les éléments suivants :
# === Les seules choses à changer sont :
#   - gcm_model    : nom du modèle global ( cnrm_cerfacs_cm5  , ichec_ec_earth  )
#   - rcm_model   :  nom du modèle régional (knmi_racmo22e ,  knmi_racmo22e )
#   - experiment  : nom du scénario (ex : 'rcp_4_5', 'rcp_8_5') 
# A RETENIR le dossier destination folder : le dossier qui va contenir les fichiers netcf du modèle futur
# Tout le reste du code doit rester inchangé.



import cdsapi
import os
import time
import sys

sys.modules["ipywidgets"] = None
destination_folder = "D:/cordex_data/donnees_futures"      # à VOIR
os.makedirs(destination_folder, exist_ok=True)


client = cdsapi.Client()


dataset = "projections-cordex-domains-single-levels"
experiment = "rcp_4_5"                                             # à MODIFIER
gcm_model = "cnrm_cerfacs_cm5"                                    # à MODIFIER                              
rcm_model = "cnrm_aladin63"                                       # à MODIFIER
ensemble_member = "r1i1p1"
horizontal_resolution = "0_11_degree_x_0_11_degree"
temporal_resolution = "daily_mean"


variables = [
    "2m_relative_humidity",
    "10m_wind_speed",
    "maximum_2m_temperature_in_the_last_24_hours",
    "minimum_2m_temperature_in_the_last_24_hours",
    "mean_precipitation_flux",
    "surface_solar_radiation_downwards"
]


periods = [
    ("2006", "2010"), ("2011", "2015"), ("2016", "2020"),
    ("2021", "2025"), ("2026", "2030"), ("2031", "2035"),
    ("2036", "2040"), ("2041", "2045"), ("2046", "2050"),
    ("2051", "2055"), ("2056", "2060"), ("2061", "2065"),
    ("2066", "2070"), ("2071", "2075"), ("2076", "2080"),
    ("2081", "2085"), ("2086", "2090"), ("2091", "2095"),
    ("2096", "2100"),
]


for i, (start_year, end_year) in enumerate(periods):
    filename = f"{destination_folder}/cordex_{experiment}_{gcm_model}_{rcm_model}_{start_year}_{end_year}.zip"
    print(f"\nBloc {i+1}/{len(periods)} : {start_year}-{end_year}")

   
    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        print(f"Déjà téléchargé : {filename}")
        continue

    print("Téléchargement en cours...")

    request = {
        "domain": "europe",
        "experiment": experiment,
        "horizontal_resolution": horizontal_resolution,
        "temporal_resolution": temporal_resolution,
        "variable": variables,
        "gcm_model": gcm_model,
        "rcm_model": rcm_model,
        "ensemble_member": ensemble_member,
        "start_year": [start_year],
        "end_year": [end_year],
        "format": "zip"
    }

    try:
        result = client.retrieve(dataset, request)
        print("Téléchargement terminé. Sauvegarde du fichier...")
        path = result.download(filename)
        print(f"Fichier enregistré : {path}")
        time.sleep(1)
    except Exception as e:
        print(f"Erreur pour {start_year}-{end_year} : {e}")

print("\nTous les blocs RCP 4.5 ont été traités avec succès.")